## Implement Data Monitors

In [49]:
# Imports
import boto3
import botocore
import json
from sagemaker import get_execution_role, Session
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.model_monitor import DefaultModelMonitor, CronExpressionGenerator, DataCaptureConfig

In [50]:
# Initialize session and role
session = Session()
role = get_execution_role()
region = 'us-east-1'
bucket = 'sagemaker-us-east-1-531690656306'

In [54]:
# Verify that endpoint exist
response = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print(f"Endpoint '{endpoint_name}' status: {status}")

Endpoint 'cardio-logistic-monitor-endpoint' status: InService


### Create Data Monitor Schedule

In [55]:
# Create monitor object
monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1200,  # Safe shorter run time
    sagemaker_session=session
)

# S3 Paths
baseline_output_uri = f's3://sagemaker-us-east-1-531690656306/cardio_data/baseline-results'
monitor_output_uri = f's3://sagemaker-us-east-1-531690656306/cardio_data/monitoring/reports'

# Monitoring schedule
schedule_name = 'cardio-data-monitor-schedule'

monitor.create_monitoring_schedule(
    monitor_schedule_name=schedule_name,
    endpoint_input=endpoint_name,
    output_s3_uri=monitor_output_uri,
    statistics=baseline_output_uri + '/statistics.json',
    constraints=baseline_output_uri + '/constraints.json',
    schedule_cron_expression=CronExpressionGenerator.daily(),
    enable_cloudwatch_metrics=True
)

print(f"Data monitor schedule '{schedule_name}' successfully created.")

Data monitor schedule 'cardio-data-monitor-schedule' successfully created.


## Infrastructure Monitoring

In [56]:
# Initialize CloudWatch client
cloudwatch = boto3.client('cloudwatch', region_name='us-east-1')
variant_name = 'AllTraffic'

# Helper function to create alarms
def create_alarm(metric_name, threshold, comparison='GreaterThanThreshold'):
    alarm_name = f"{endpoint_name}-{metric_name}-Alarm"

    response = cloudwatch.put_metric_alarm(
        AlarmName=alarm_name,
        MetricName=metric_name,
        Namespace='AWS/SageMaker',
        Dimensions=[
            {'Name': 'EndpointName', 'Value': endpoint_name},
            {'Name': 'VariantName', 'Value': variant_name}
        ],
        Statistic='Average',
        Period=300,
        EvaluationPeriods=1,
        Threshold=threshold,
        ComparisonOperator=comparison,
        ActionsEnabled=False,
        AlarmDescription=f"Alarm when {metric_name} is above {threshold}%"
    )
    print(f"Alarm created for {metric_name}")

# Create CPU, Memory, Disk alarms
create_alarm('CPUUtilization', threshold=70)
create_alarm('MemoryUtilization', threshold=75)
create_alarm('DiskUtilization', threshold=80)

Alarm created for CPUUtilization
Alarm created for MemoryUtilization
Alarm created for DiskUtilization


### Save files into s3 bucket

In [59]:
# Save notebook (.ipynb)
!aws s3 cp cardio_model_monitoring.ipynb s3://{bucket}/{folder}cardio_data_and_infrasturure_monitors.ipynb

upload: ./cardio_model_monitoring.ipynb to s3://sagemaker-us-east-1-531690656306/cardio_project/cardio_data_and_infrasturure_monitors.ipynb
